In [ ]:
case class OrderLine (
    sku : String,
    productName : String,
    thumbnailImage: String,
    quantity : Double,
    unitPrice : Double,
    totalPrice : Double
)

case class Order (
    customerId : java.util.UUID ,
    orderId : java.util.UUID ,
    date : java.util.Date ,
    OrderLines_ : List[OrderLine] ,
    totalPrice : Double
)

case class OrderLines (
    OrderLines_ : List[OrderLine]
)

case class RecommendedProduct (
    sku : String,
    product_name : String,
    regular_price : Double,
    thumbnail_image : String
);

case class ProductRecommendations (
    sku : String,
    product_name : String,
    recommended_products : List[RecommendedProduct]
);

case class Top50SellingProducts (
    sku : String,
    productName : String,
    saleCount : Double,
    saleValue : Double,
    thumbnailImage : String
);



In [ ]:
val orders = sc.cassandraTable[Order]("retail_ks","orders")

In [ ]:
val orders = sc.cassandraTable[OrderLines]("retail_ks","orders").select("order_lines_")

In [ ]:
orders.count

In [ ]:
val orderlines = orders.flatMap ( order => order.OrderLines_).map( ol => (ol.sku, (ol.productName, ol.thumbnailImage, ol.quantity, ol. unitPrice, ol.totalPrice)))

In [ ]:
orderlines.take(10)

In [ ]:
val soldproducts = orderlines.reduceByKey( (a,b) => (a._1, a._2, a._3+b._3, a._4, a._5+b._5)).
    map( { case ( sku, (productName, thumbnailImage, count, unitPrice, value) ) => Top50SellingProducts (sku, productName, count, value, thumbnailImage) })

In [ ]:
val Top50CountSellingProducts = soldproducts.sortBy(  -_.saleValue  ).
    zipWithIndex.
    filter{case (_, idx) => idx < 50}.
    keys

In [ ]:
Top50CountSellingProducts.take(100)

In [ ]:
Top50CountSellingProducts.saveToCassandra("retail_ks","top50_selling_products" )

In [ ]:
val ProductCoOccurance = orders.
    flatMap( order => order.OrderLines_.
        map(  ol => ( ol.sku , order.OrderLines_.
            map(ol => (ol.sku, (ol.productName, ol.thumbnailImage, ol.quantity, ol. unitPrice, ol.totalPrice))).filter ( ol3 => ol3._1!=`ol`.sku)
                ) // for each order, make a list of product cooccurance (product1, product2)
            )
            ).reduceByKey{ (a,b) =>
                          // merge the list per product1
                          val mergedBySku = (a++b)
                          // for each product1, merge list on key product2
                          val groupedBySku = mergedBySku.groupBy( { case (sku,(pn, ti, q, up, tp)) => sku } ).values.toList
                          // aggregate value sum of product2
                          groupedBySku.map( listOfProduct => listOfProduct.reduce( (a,b) => (a._1, (a._2._1, a._2._2, a._2._3+b._2._3, a._2._4, a._2._5+b._2._5))) )
                           }.
                mapValues { TotalSumBySku =>
                // take top50 product2 sorted on summed value
                val Top50Value = TotalSumBySku.sortBy(-_._2._5).slice(0,50)
                // create a RecommendedProduct item to fit table structure
                Top50Value.map( lop => RecommendedProduct(lop._1, lop._2._1, lop._2._4  , lop._2._2  ))
                        }.
            // create a ProductRecommendation to fit table structure
            map( pco => ProductRecommendations(pco._1, "", pco._2))

In [ ]:
ProductCoOccurance.take(1)

In [1]:
val sqlContext = new org.apache.spark.sql.hive.HiveContext(sc)
import sqlContext.implicits._

In [ ]:
val pcodf = ProductCoOccurance.toDF
pcodf.printSchema

In [ ]:
pcodf.write.format("org.apache.spark.sql.cassandra").
    options(Map( "table" -> "product_recommendations", "keyspace" -> "retail_ks")).
    mode("overwrite").
    save()


In [ ]:
ProductCoOccurance.saveToCassandra("retail_ks","product_recommendations")

In [ ]:
val pcodf_read = sqlContext.read.format("org.apache.spark.sql.cassandra").
	options(Map( "table" -> "product_recommendations", "keyspace" -> "retail_ks")).
	load()


In [ ]:
pcodf_read.count

In [ ]:
sc.cassandraTable("retail_ks","product_catalog").count

In [ ]:
sc.cassandraTable("retail_ks","product_accessories").count

In [ ]:
val orders_df = sqlContext.read.format("org.apache.spark.sql.cassandra").
	options(Map( "table" -> "orders", "keyspace" -> "retail_ks")).
	load()

In [ ]:
orders_df.printSchema

In [3]:
sqlContext.read.format("org.apache.spark.sql.cassandra").options(Map( "table" -> "orders", "keyspace" -> "retail_ks")).load().
registerTempTable("orders")
val top50MostSoldProducts = sqlContext.sql("SELECT ol.sku, sum(ol.total_price) as sale_value FROM "+
"(SELECT explode(order_lines_) as ol FROM orders) ols group by ol.sku order by sale_value DESC limit 50")
top50MostSoldProducts.show(100)

+-------+--------------------+
|    sku|          sale_value|
+-------+--------------------+
|7739048| 4.841700316580001E9|
|4920300|3.9359501601000004E9|
|3429088| 3.508382976019998E9|
|5182036|1.9988070029700005E9|
|5034500|1.8577451580799997E9|
|3892044|      1.7299176147E9|
|5034600|     1.69519761411E9|
|5035005|1.5128629141799996E9|
|5042100|1.4186518666999993E9|
|5418101|1.2521674913100004E9|
|3429111|     1.24604751579E9|
|4506048|     1.16833512026E9|
|5065700|     1.16065775364E9|
|6273121|     1.13445133512E9|
|4507056|1.0710284998399999E9|
|4209600|1.0486804062599998E9|
|4516300|1.0314208428699999E9|
|4575002| 9.948360515399998E8|
|5149031| 8.938876339299996E8|
|4213306| 8.171932925100001E8|
|5185184| 7.523689682799999E8|
|3953166| 6.565352496900002E8|
|5416149| 6.553015591799997E8|
|4213305|      6.4491463908E8|
|4517500| 6.116409029499999E8|
|9212012|      5.9779421834E8|
|3953148|      5.8863829464E8|
|3902015| 5.430073269600002E8|
|4391930|5.0294856300000083E8|
|5412152

In [ ]:
top50MostSoldProducts.write.format("org.apache.spark.sql.cassandra").
options(Map( "table" -> "top50_selling_products", "keyspace" -> "retail_ks")).
mode("overwrite").    
save()